<a href="https://colab.research.google.com/github/dorelysm/NLP_master/blob/main/caracteristicasOdio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Universidad Internacional de La Rioja (UNIR) - Máster Universitario en Inteligencia Artificial - Procesamiento del Lenguaje Natural**

***
Datos del alumno (Nombre y Apellidos): Dorelys Esthafany Martinez Suarez

Fecha: 03/02/25
***

<span style="font-size: 20pt; font-weight: bold; color: #0098cd;">Trabajo: Caracterización de textos</span>

**Objetivos**

Con esta actividad se tratará de que el alumno se familiarice con el manejo de la librería spacy, así como con los conceptos básicos de manejo de las técnicas NER

**Descripción**

En esta actividad debes procesar de forma automática un texto en lenguaje natural para detectar características básicas en el mismo, y para identificar y etiquetar las ocurrencias de conceptos como localización, moneda, empresas, etc.

En la primera parte del ejercicio se proporciona un código fuente a través del cual se lee un archivo de texto y se realiza un preprocesado del mismo. En esta parte el alumno tan sólo debe ejecutar y entender el código proporcionado.

En la segunda parte del ejercicio se plantean una serie de preguntas que deben ser respondidas por el alumno. Cada pregunta deberá responderse con un fragmento de código fuente que esté acompañado de la explicación correspondiente. Para elaborar el código solicitado, el alumno deberá visitar la documentación de la librería spacy, cuyos enlaces se proporcionarán donde corresponda.

# Parte 1: carga y preprocesamiento del texto a analizar

Observa las diferentes librerías que se están importando.

In [2]:
!python -m spacy download es_core_news_md

  Using cached https://github.com/explosion/spacy-models/releases/download/es_core_news_md-3.8.0/es_core_news_md-3.8.0-py3-none-any.whl (42.3 MB)
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
import pathlib
import spacy
import pandas as pd
from spacy import displacy
import csv
import es_core_news_md

El siguiente código simplemente carga y preprocesa el texto. Para ello, lo primero que hace es cargar un modelo de lenguaje previamente entrenado. En este caso, se utiliza <i>es_core_news_md</i>:

https://spacy.io/models/es#es_core_news_md


In [4]:
nlp = es_core_news_md.load()

El objeto <i>nlp</i> permite utilizar el modelo de lenguaje cargado, de forma que se puede procesar un texto y obtenerlo en su versión preprocesada. Así, nos permite realizar las diferentes tareas. En este caso, vamos a utilizar el pipeline para hacer un preprocesamiento básico, que consiste en tokenizar el texto.

In [6]:
from pathlib import Path

In [14]:
# Rutas de entrada/salida (ajustad el nombre si hace falta)

input_path = Path("./02Dataset_anonimizado.csv")  # fichero original

tmp_path = Path("comentarios_sin_bytes_raros.csv")

output_path = Path("comentarios_limpio_utf8.csv")

# Algunos bytes problemáticos detectados en el archivo original

BAD_BYTES = {0xBF, 0xA1, 0xB3}  # ¿, ¡, ³ en cp1252

# 1) Eliminamos esos bytes a nivel binario

with input_path.open("rb") as f_in, tmp_path.open("wb") as f_out:

    for raw_line in f_in:

        cleaned = bytes(b for b in raw_line if b not in BAD_BYTES)

        f_out.write(cleaned)
# 2) Leemos el CSV resultante asumiendo codificación latin1 / cp1252

df = pd.read_csv(tmp_path, sep=";", encoding="latin1")



# 3) Arreglamos el "mojibake" típico (debÃ­a -> debía, etc.)

def arreglar_mojibake(s):

    if not isinstance(s, str):

        return s

    try:

        reparado = s.encode("latin1").decode("utf-8")

        return reparado

    except UnicodeError:

        return s



text_cols = df.select_dtypes(include="object").columns

for col in text_cols:

    df[col] = df[col].apply(arreglar_mojibake)



# 4) Guardamos ya todo en UTF-8 limpio

df.to_csv(output_path, sep=";", index=False, encoding="utf-8")





print("Archivo limpio guardado como:", output_path)

/tmp/ipython-input-3024919559.py:24: DtypeWarning: Columns (6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(tmp_path, sep=";", encoding="latin1")


Archivo limpio guardado como: comentarios_limpio_utf8.csv


In [17]:
lines_number = 10000

data = pd.read_csv("comentarios_limpio_utf8.csv", sep=";", encoding="utf-8", nrows=lines_number)


El código anterior carga el archivo CSV (opcionalmente con un límite de líneas a leer) y genera la variable <i>data</i>, que contiene un Dataframe (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) con los datos leídos del CSV.

Te vendrá bien conocer la siguiente documentación:
<ul>
    <li>https://spacy.io/api/doc</li>
    <li>https://spacy.io/api/token</li>
    <li>https://spacy.io/api/morphology#morphanalysis</li>
</ul>

### Playground

Utiliza este espacio para hacer pruebas y ensayos con las variables generadas con el código previo. A modo de ejemplo, se ofrece código que realiza las siguientes tareas:


- leer un número dado de líneas del Dataframe y generar dos listas con los valores (se pueden leer directamente del DataFrame, se muestra el ejemplo como una opción más)
- procesar el texto de cada comentario


Para procesarlo, hay utilizar el objeto <i>nlp</i> y así obtener objetos de la clase <i>Doc</i> (https://spacy.io/api/doc)

Visita la documentación de dicha clase y experimenta probando las diferentes funciones y atributos

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   MEDIO                 10000 non-null  object 
 1   SOPORTE               10000 non-null  object 
 2   URL                   10000 non-null  object 
 3   TIPO DE MENSAJE       10000 non-null  object 
 4   CONTENIDO A ANALIZAR  10000 non-null  object 
 5   INTENSIDAD            10000 non-null  float64
 6   TIPO DE ODIO          748 non-null    object 
 7   TONO HUMORISTICO      23 non-null     object 
 8   MODIFICADOR           36 non-null     object 
 9   Unnamed: 9            0 non-null      float64
 10  Unnamed: 10           0 non-null      float64
 11  Unnamed: 11           0 non-null      float64
 12  Unnamed: 12           0 non-null      float64
 13  Unnamed: 13           0 non-null      float64
 14  Unnamed: 14           0 non-null      float64
 15  Unnamed: 15         

In [20]:
data.head(5)

,MEDIO,SOPORTE,URL,TIPO DE MENSAJE,CONTENIDO A ANALIZAR,INTENSIDAD,TIPO DE ODIO,TONO HUMORISTICO,MODIFICADOR,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,EL PAÍS,WEB,URL_a4d7efc0,COMENTARIO,el barí§a nunca acaeza ante un segundo b ni an...,3.0,Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EL PAÍS,WEB,URL_a4d7efc0,COMENTARIO,el real madrid ha puesto punto y final a su an...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EL PAÍS,WEB,URL_54312d9e,COMENTARIO,cristina cifuentes podrí­a haber sido la presi...,3.0,Ideológico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EL PAÍS,WEB,URL_54312d9e,COMENTARIO,habrí­a que reabrir el caso. el supremo se ded...,3.0,Ideológico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EL PAÍS,WEB,URL_54312d9e,COMENTARIO,me parece un poco exagerado pedir más de tres ...,3.0,Ideológico,Si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
data.describe()

,INTENSIDAD,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
count,10000.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,0.252600,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.944289,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,6.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# Puedes insertar aquí código de pruebas para experimentar con las diferentes funciones y atributos de 'doc'.
#print(data["CONTENIDO A ANALIZAR"][1])
#print(data["INTENSIDAD"][1])
doc = []
value = []

#con el bucle, generamos sendas listas con los comentarios ya parseados y con el valor de intensidad
for i in range(0, lines_number):

    #en un primer paso se parsea el comentario. En el segundo paso se añade el objeto a la lista
    tmp_doc = nlp(data["CONTENIDO A ANALIZAR"][i])
    doc.append(tmp_doc)

    #en un primer paso extrae el valor. En el segundo paso se añade el valor a la lista
    tmp_value = data["INTENSIDAD"][i]
    value.append(tmp_value)


#ejemplo de cómo recorrer un comentario palabra por palabra
for token in doc[1]:
    print(token)

el
real
madrid
ha
puesto
punto
y
final
a
su
andadura
en
la
copa
del
rey
en
el
primer
escalón
.
los
de
zidane
han
caí­do
ante
el
alcoyano
,
de
segunda
b
,
a
pesar
de
empezar
ganando
y
jugar
con
un
hombre
menos
en
la
prórroga
.
el
técnico
francés
dispuso
un
equipo
plagado
de
los
menos
habituales
,
con
vinicius
y
mariano
en
ataque
.
ninguno
de
los
dos
logró
crear
ocasiones
.
fue
militao
el
que
marcó
el
gol
del
madrid
,
justo
antes
del
descanso
.
en
la
segunda
parte
intentaron
cerrar
el
partido
,
pero
sin
el
colmillo
suficiente
y
el
modesto
alcoyano
aprovechó
un
córner
para
empatar
el
partido
a
cinco
minutos
para
el
final
.
el
empate
sentó
como
un
jarro
de
agua
frí­a
a
los
blancos
,
que
lo
intentaron
en
el
tiempo
extra
a
falta
de
cinco
minutos
,
el
casanova
consiguió
el
gol
más
importante
de
su
vida
,
que
vale
la
clasificación
para
octavos
de
la
copa
.
el
madrid
de
zidane
queda
apeado
del
torneo
una
vez
más
,
por
lo
que
el
francés
se
quedará
sin
pelear
por
el
único
tí­tulo
que
no
ha
conseg

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Pregunta 1.</span>
<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">¿Cuántos registros contiene el corpus?</span>

In [24]:
# Incluye aquí el código generado para poder responder a tu pregunta
n_registros=(data.shape[0])
print(n_registros)

10000


<b>Incluye aquí, debajo de la línea, la explicación de tu respuesta</b>
<hr>
 El corpus tiene 10000 registros que son los que se leyeron del archivo .csv al pedirle un lines_number de 10000.
 Utilicé la funcion shape para obtener el número de registros

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Pregunta 2.</span>
<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">¿Cuántas palabras totales hay en los comentarios del corpus?</span>

In [27]:
# Incluye aquí el código generado para poder responder a tu pregunta
data["palabras"]=data["CONTENIDO A ANALIZAR"].str.split().str.len()
suma=data["palabras"].sum()
print(suma)

216760


<b>Incluye aquí, debajo de la línea, la explicación de tu respuesta</b>
<hr>
 El total de palabras que hay los comentarios del corpus es 216760.
 Para obtener este dato, creé una nueva columna "palabras" y en ella guardé el resultado de cuantas palabras hay en el comentario "CONTENIDO A ANALIZAR" de cada fila y finalmente guardé una variable "suma" con el resultado de sumar todos los datos de la columna "palabras"

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Pregunta 3.</span>
<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">¿Cuál el número promedio de palabras en cada comentario?</span>

In [29]:
# Incluye aquí el código generado para poder responder a tu pregunta
promedio_palabras=suma/n_registros
print(promedio_palabras)

21.676


<b>Incluye aquí, debajo de la línea, la explicación de tu respuesta</b>
<hr>
El promedio de palabras en cada comentario del corpus es de 21.676 palabras por comentario.
Esta respuesta la obtuve de dividir la variable "suma" entre la variable "n_registros", estas 2 variables fueron obtenidas de las operaciones para hallar las respuestas a las preguntas 1 y 2.


<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Pregunta 4.</span>
<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Considerando dos grupos de comentarios (odio y no odio) ¿Cuál el número promedio de palabras en los comentarios de cada grupo?</span>

In [30]:
# Incluye aquí el código generado para poder responder a tu pregunta

promedios_por_intensidad = data.groupby('INTENSIDAD').agg({'palabras': 'mean'})
promedios_por_intensidad



,palabras
INTENSIDAD,
0.0,21.575875
1.0,29.794393
2.0,62.160714
3.0,38.794872
4.0,13.959052
5.0,22.292683
6.0,27.000000


In [31]:
data["Odio"] = data["INTENSIDAD"].apply(lambda x: "no odio" if x == 0.0 else "odio")
data.head()

,MEDIO,SOPORTE,URL,TIPO DE MENSAJE,CONTENIDO A ANALIZAR,INTENSIDAD,TIPO DE ODIO,TONO HUMORISTICO,MODIFICADOR,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,palabras,Odio
0,EL PAÍS,WEB,URL_a4d7efc0,COMENTARIO,el barí§a nunca acaeza ante un segundo b ni an...,3.0,Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25,odio
1,EL PAÍS,WEB,URL_a4d7efc0,COMENTARIO,el real madrid ha puesto punto y final a su an...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,195,no odio
2,EL PAÍS,WEB,URL_54312d9e,COMENTARIO,cristina cifuentes podrí­a haber sido la presi...,3.0,Ideológico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,127,odio
3,EL PAÍS,WEB,URL_54312d9e,COMENTARIO,habrí­a que reabrir el caso. el supremo se ded...,3.0,Ideológico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,odio
4,EL PAÍS,WEB,URL_54312d9e,COMENTARIO,me parece un poco exagerado pedir más de tres ...,3.0,Ideológico,Si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,147,odio


In [32]:
data.groupby('Odio').agg({'palabras': 'mean'})

,palabras
Odio,
no odio,21.575875
odio,22.914439


<b>Incluye aquí, debajo de la línea, la explicación de tu respuesta</b>
<hr>
 En promedio los comentarios de no odio tienen un promedio de 21.57 palabras y los de odio tienen un promedio de 22.91 palabras.
 Para llegar a esta respuesta, ya que la característica odio está relacionada con la columna INTENSIDAD, primero calculé los promedios de palabras agrupados por la colimna INTENSIDAD, en donde se muestra que para esta columna los datos van de 0 a 6, consideré que 0 equivale a no odio y los demás grupos equivalen a odio, utilicé una funcion lambda para crear una nueva columna llamada "odio" y asignarle los valores "odio" u "no odio" dependiendo del valor de intensidad correspondiente.
 Finalmente calculé los promedios de palabras agrupados por la columna odio.


<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Pregunta 5.</span>
<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Considerando dos grupos de comentarios (odio y no odio) ¿Cuál es el número promedio de oraciones en los comentarios de cada grupo?</span>

In [38]:
# Incluye aquí el código generado para poder responder a tu pregunta
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [39]:
data["n_oraciones"] = data["CONTENIDO A ANALIZAR"].astype(str).apply(lambda x: len(sent_tokenize(x)))

In [40]:
data.groupby('Odio').agg({'n_oraciones': 'mean'})

,n_oraciones
Odio,
no odio,1.774643
odio,1.798128


<b>Incluye aquí, debajo de la línea, la explicación de tu respuesta</b>
<hr>
 En promedio los comentarios de no odio contienen 1.77 oraciones y los de odio 1.98 oraciones.
 Para obtener estos datos necesité utilizar la librería nltk para el procesamiento de lenguaje natural y la función sent_tokenize de esta librería para realizar la tokenización de cada comentario "CONTENIDO A ANALIZAR" y así un resultado aproximado a la cantidad de oraciones de los comentarios. El resultado de cada fila lo guardé en una nueva columna llamada "n_oraciones".

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Pregunta 6.</span>
<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Considerando dos grupos de comentarios (odio y no odio) ¿Cuál es el porcentaje de comentarios que contienen entidades NER en cada grupo?</span>

In [44]:
# Incluye aquí el código generado para poder responder a tu pregunta
data["NER"]= data["CONTENIDO A ANALIZAR"].apply(lambda x: nlp(x).ents)
data.head()

,MEDIO,SOPORTE,URL,TIPO DE MENSAJE,CONTENIDO A ANALIZAR,INTENSIDAD,TIPO DE ODIO,TONO HUMORISTICO,MODIFICADOR,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,palabras,Odio,n_oraciones,NER
0,EL PAÍS,WEB,URL_a4d7efc0,COMENTARIO,el barí§a nunca acaeza ante un segundo b ni an...,3.0,Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25,odio,1,"((messi),)"
1,EL PAÍS,WEB,URL_a4d7efc0,COMENTARIO,el real madrid ha puesto punto y final a su an...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,195,no odio,9,"((real, madrid), (copa, del, rey), (zidane), (..."
2,EL PAÍS,WEB,URL_54312d9e,COMENTARIO,cristina cifuentes podrí­a haber sido la presi...,3.0,Ideológico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,127,odio,7,"((cristina, cifuentes), (madrid), (pp), (madrid))"
3,EL PAÍS,WEB,URL_54312d9e,COMENTARIO,habrí­a que reabrir el caso. el supremo se ded...,3.0,Ideológico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,odio,3,()
4,EL PAÍS,WEB,URL_54312d9e,COMENTARIO,me parece un poco exagerado pedir más de tres ...,3.0,Ideológico,Si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,147,odio,6,"((casadete), (harvard), (aravaca), (costa, de,..."


In [45]:
data["tiene_NER"]= data["NER"].apply(lambda x: len(x)>0)
data.head()

,MEDIO,SOPORTE,URL,TIPO DE MENSAJE,CONTENIDO A ANALIZAR,INTENSIDAD,TIPO DE ODIO,TONO HUMORISTICO,MODIFICADOR,Unnamed: 9,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,palabras,Odio,n_oraciones,NER,tiene_NER
0,EL PAÍS,WEB,URL_a4d7efc0,COMENTARIO,el barí§a nunca acaeza ante un segundo b ni an...,3.0,Otros,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,25,odio,1,"((messi),)",True
1,EL PAÍS,WEB,URL_a4d7efc0,COMENTARIO,el real madrid ha puesto punto y final a su an...,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,195,no odio,9,"((real, madrid), (copa, del, rey), (zidane), (...",True
2,EL PAÍS,WEB,URL_54312d9e,COMENTARIO,cristina cifuentes podrí­a haber sido la presi...,3.0,Ideológico,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,127,odio,7,"((cristina, cifuentes), (madrid), (pp), (madrid))",True
3,EL PAÍS,WEB,URL_54312d9e,COMENTARIO,habrí­a que reabrir el caso. el supremo se ded...,3.0,Ideológico,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,19,odio,3,(),False
4,EL PAÍS,WEB,URL_54312d9e,COMENTARIO,me parece un poco exagerado pedir más de tres ...,3.0,Ideológico,Si,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,147,odio,6,"((casadete), (harvard), (aravaca), (costa, de,...",True


In [46]:
porcentaje_de_NER=data["tiene_NER"].mean()*100
print(porcentaje_de_NER)

34.35


<b>Incluye aquí, debajo de la línea, la explicación de tu respuesta</b>
<hr>
El porcentaje de comentarios que tienen entidades NER es de 34.35%.
Para obtener este dato, primero calculé y guardé en una nueva columna "NER" las entidades NER de cada comentario. Después, calculé si la longitud de los valores de cada fila de la columna NER eran mayores que cero  y guardé los resultados en una nueva columna "tiene_NER". En pandas true equivale a 1 y false a 0, entonces finalmente calculé el promedio de la columna "tiene_NER" para obtener así la respuesta.

<span style="font-size: 16pt; font-weight: bold; color: #0098cd;">Plantea tus propias preguntas</span>

<span><b>Plantea al menos 4 características</b> del texto cuyo análisis permita una caracterización completa del texto. Puedes utilizar recomendaciones proporcionadas por la IA Generativa, si así lo deseas. Para cada una de las características planteadas, obtén valores separados para los grupos ODIO/NO-ODIO.</span>

<span>En la explicación aportada, deberás <b>explicar el significado de la característica planteada</b> así como la importancia de ésta en la caracterización del texto.</span>

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Característica adicional 1.</span>


<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Uso de signos de exclamación</span>

In [53]:
# Incluye aquí el código generado para poder responder a tu pregunta

# Conteo de signos de exclamación
def count_exclamations(text):
    if pd.isna(text):
        return 0
    return text.count("!")

data["exclamaciones"] = data["CONTENIDO A ANALIZAR"].apply(count_exclamations)

data.groupby('Odio').agg({'exclamaciones': 'mean'})

,exclamaciones
Odio,
no odio,0.247190
odio,0.360963


<b>Incluye aquí, debajo de la línea, la explicación de la característica propuesta y su motivación. Incluye también una explicación del código fuente aportado.</b>
<hr>
Considerando que muchos personas suelen levantar la voz o gritar al expresar odio, realicé un análisis de cuantos signos de exclamación "!" hay por cada grupo "no odio" y "odio", encontrando que se utilizan más de estos signos en los comentarios de odio.
Para el análisis conté los signos "!" de cada comentario y guardé los resultados en una nueva columna "explamaciones", luego utilicé groupby para agrupar por la columna "Odio" y mostrar el promedio de signos de exclamación en "no odio" y "odio"

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Característica adicional 2.</span>


<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Porcentaje de palabras ofensivas</span>

In [59]:
# Incluye aquí el código generado para poder responder a tu pregunta

#Lexicon generado con Inteligencia Artificial
lexicon_ofensivo = {
    # Insultos generales
    "idiota","imbecil","imbécil","estupido","estúpido","tarado","tonto","bobo","bobolon","baboso","babosa",
    "retardado","retrasado","mongolico","mongólico","subnormal","anormal","bestia","animal","bruto","burro",
    "payaso","payasa","payasito","payasita","estupida","pendejo","pendeja","pelotudo","pelotuda","boludo","boluda",
    "huevon","huevón","huevona","hueva","guevon","guevón","guevona",

    # Vulgaridades fuertes
    "mierda","mierdero","mrd","mrda","puto","puta","putita","putazo","putear","putearon","putearte","putearlo",
    "putero","putilla","cabron","cabrón","cabrona","cabronazo","cabroncete","cabrón","chingada","chingado","chingar",
    "chingada","chingón","chingona","joder","jodido","jodida","carajo","cojudo","cojuda","coño","culero","culera",
    "culo","culiao","culiá","culiaron","culiando","fuck","fucking","hpta","hp","hijueputa","hijueputas","hijueputica",
    "malparido","malparida","maldito","maldita","mamón","mamona","mamerto","cagon","cagón","cagona","cagar","cagada",
    "zorra","perra","perrita","perraca","malnacido","malnacida",

    # Insultos físicos / mentales
    "feo","fea","gordo","gorda","cerdo","cerda","chancho","chancha","asqueroso","asquerosa","repugnante","apestoso",
    "mugroso","mugrosa","sucio","sucia","rata","ratón","ratera","ratero","parásito","inutil","inútil","vago","vaga",
    "quita","pargo","larva","gusano","gusana","holgazán","holgazana","flojo","floja",

    # Sexistas / Misóginos
    "feminazi","loca","zorra","arpía","locaza","puta","putilla","buscona","arrastrada","mosquita","golfa","ramera",
    "marimacha","vividora","lagarta","mosquita","zanganona",

    # Homofobia
    "marica","marico","marikon","maricón","maricona","mariconazo","gayazo","loca","locote","puñal","travesti",
    "trava","machorra","sarasa","puto","putito",

    # Xenofobia / desprecio por nacionalidades
    "sudaca","indio","india","muerto de hambre","moro","morito","moraca","negro","negra","negrata",
    "cholo","chola","gringo","gringa","panchito","panchita","gallego","gallega",

    # Racismo explícito
    "simio","mono","primate","esclavo","bestia","cimarron","cimarrón","marrano","bestia",

    # Desprecio social / intelectual
    "ignorante","inculto","cenutrio","miserable","patético","patetico","fracasado","fracasada","perdedor","basura",
    "chusma","lacra","escoria","paria","delincuente","vándalo","vandalos","ocioso","payasería","payaserias",

    # Aversión política (típicos insultos)
    "facha","fachoso","fachito","socialista de mierda","comunista de mierda",
    "uribista hijueputa","mamerto","mamertos","castrochavista","uribestia","petrista hijueputa",

    # Variaciones con símbolos / estilizaciones
    "p3ndejo","p3ndeja","pndjo","idi0ta","estvpido","imbecil","imbécil","mrda","mrd","pndja",
    "pvt0","pvto","pvta","hpt","hpta","hp","kbron","kbrn","pendjo","p3nda","m13rda","m1erda",

    # Ofensas coloquiales latinoamericanas
    "careverga","caremonda","carechimba","chimbo","gonorrea","gonorreas","gono","güevon","guevon","guevón",
    "marimonda","ñero","ñera","ñor","guisa","hijueperra","hijueperro","perro","perra","pirobo","piroba",
    "piroberto","sapoperro","zarrapastroso","zarrapastrosa","vago","zangano","zangana","paraco","guerrillero de mierda",

    # Más variaciones para robustez
    "imbesil","invutile","inutil","inepto","idiotazo","idiotaza","cretino","cretina","tarugo","taruga","mamerto",
    "mierdero","culoncito","culona","culon","maldito","maldita","ojete","picha","pichurria","verga","vergazo",
    "verguita","pelmazo","pelmaza","babosada","sandeces","bastardo","bastarda","zopenco","zopenca",

    # Insultos hacia grupos
    "nazi","hembrista","machista de mierda","femiloca","paraco hijueputa","sapo","sapa","metido","chismoso","chismosa",

    # Ofensas sobre inteligencia
    "cerebro de mosquito","cabeza hueca","sin cerebro","descerebrado","descerebrada","torpe","zoquete","lelo","lela",

    # Acoso / degradación sexual
    "culicagado","culicagada","pelagato","prostituta","prostituto","maleducado","maleducada","lambón","lambona",
    "arrastrado","arrastrada","asquerosidad","inmundicia",

    # Otros ofensivos comunes
    "puerco","porquería","porqueria","apestoso","apestosa","apestosos","apestosas","amargado","amargada",
    "infeliz","torcido","torcida","intenso","imbecilote","plasta"
}


def n_de_palabras_ofensivas(comentario, lexicon_ofensivo):
  if len(comentario) == 0:
    return 0

  ofensivas = sum(1 for palabra in comentario.split() if palabra.lower() in lexicon_ofensivo)
  return ofensivas

data["n_palabras_ofensivas"] = data["CONTENIDO A ANALIZAR"].apply(lambda x: n_de_palabras_ofensivas(x, lexicon_ofensivo))

data.groupby('Odio').agg({'porcentaje_palabras_ofensivas': 'mean'})


,porcentaje_palabras_ofensivas
Odio,
no odio,0.098927
odio,0.572236


<b>Incluye aquí, debajo de la línea, la explicación de la característica propuesta y su motivación. Incluye también una explicación del código fuente aportado.</b>
<hr>
Elegí la característica de porcentaje de palabras ofensivas para analizar cúal era la diferencia en el uso de estas entre los comentarios de odio y no odio, encontrandome que el 57% de los comentarios de odio las contienen vs el 9% de los de no odio.
Para hacer el análisis, primero necesité un listado de palabras de odio con las cuales buscar luego en los comentarios, así que generé este lexicon con el uso de IA.
Luego, en una nueva columna "n_palabras_ofensivas" calculé y guardé el número de palabras ofensivas que encontrara en el comentario de cada fila. Al final agrupé y calculé en odio y no odio el promedio de palabras ofensivas del grupo.

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Característica adicional 3.</span>


<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Sarcasmo</span>

In [67]:
import re

# Incluye aquí el código generado para poder responder a tu pregunta
MARCADORES_SARCASMO = [
    "jaja", "jajaja", "jajaj", "xd", "xD", "XD", "lol", "lmao", "rofl",
    "si claro", "sí claro", "claro que sí", "claro que si", "claro aja",
    "obvio", "seguro", "bravo", "wow", "ah bueno", "si como no", "sí como no",
    "aja", "ajá", "aha",
    "nooo para nada", "uff sí", "uff si",
    "😉","😏","🙄","😒","🤡","🤦","😂","😹","😌"
]

def sarcasmo(text):
    if pd.isna(text):
        return 0

    t = text.lower()

    score = 0

    for m in MARCADORES_SARCASMO:
        if m in t:
            score += 1

    if "???" in text or "!!!" in text:
        score += 1

    if re.search(r'\".*?\"', text):
        score += 1


    return score

data["sarcasmo"] = data["CONTENIDO A ANALIZAR"].apply(sarcasmo)

data.groupby('Odio').agg({'sarcasmo': 'mean'})

,sarcasmo
Odio,
no odio,0.179853
odio,0.192513


<b>Incluye aquí, debajo de la línea, la explicación de la característica propuesta y su motivación. Incluye también una explicación del código fuente aportado.</b>
<hr>
Analicé el nivel de sarcasmo en los comentarios porque pienso que suele utilizarse sobre todo en comentarios de odio. Para hacerlo primero generé un listado de marcadores de sarcasmo con IA, luego escribí una función para encontrar los marcadores entre las palabras de cada comentario, además detectar si hay repetición de sígnos "???" y "!!!", cada vez que se encuentra uno de lo anterior aumenta la variable score.

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Característica adicional 4.</span>


<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Proporción de adjetivos</span>

In [72]:
# Incluye aquí el código generado para poder responder a tu pregunta
def proporcion_adjetivos(text):
    if pd.isna(text):
        return 0

    doc = nlp(text)

    tokens = [token for token in doc if token.is_alpha]

    if len(tokens) == 0:
        return 0

    adjetivos = [token for token in tokens if token.pos_ == "ADJ"]

    return len(adjetivos) / len(tokens)

data["prop_adjetivos"] = data["CONTENIDO A ANALIZAR"].apply(proporcion_adjetivos)

data.groupby('Odio').agg({'prop_adjetivos': 'mean'})

,prop_adjetivos
Odio,
no odio,0.080537
odio,0.095871


<b>Incluye aquí, debajo de la línea, la explicación de la característica propuesta y su motivación. Incluye también una explicación del código fuente aportado.</b>
<hr>
Aquí analicé la proporción de adjetivos de cada comentario, si el token (palabra) corresponde a la categoría "ADJ" se van sumando y luego se divide entre el numero de tokens del comentatio.
El resultado es que los comentarios de odio tienen una proporción de adjetivos ligeramente superior que los de no odio.

<span style="font-size: 16pt; font-weight: bold; color: #0098cd;">Reflexión final</span>

<span>Una de las utilidades de la caracterización de texto es que nos sirve como etapa de <i>feature-extraction</i> (extración de características) de cara a un posterior sistema de clasificación. Es pertinente, por tanto, reflexionar sobre la capacidad discriminatoria de cada una de las características extraídas. </span>

<span> Responde, para ello, a la siguiente pregunta.</span>

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Reflexión final.</span>
<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">¿Es posible utilizar alguna de las características extraídas en las preguntas anteriores para determinar si un mensaje contiene odio? Justifica tu respuesta con el análisis estadístico que consideres necesario.</span>

In [73]:
# Incluye aquí el código generado para poder responder a tu pregunta
data.groupby('Odio').agg({'porcentaje_palabras_ofensivas': 'mean'})

,porcentaje_palabras_ofensivas
Odio,
no odio,0.098927
odio,0.572236


<b>Incluye aquí, debajo de la línea, la explicación de tu respuesta</b>
<hr>
Si, creo que es posible utilizar la característica del porcentaje de palabras ofensivas para detectar odio. Aunque en los de no odio tambien se encuentran algunas palabras ofensivas la diferencia entre ambos grupos fue de 57% a 9%, por lo que solo utilizando esta característica se podrían detectar más de la mitad de los comentarios de odio.